In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
import tensorflow.keras.losses as losses
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import tensorflow.keras.optimizers as optimizers
!pip install -q -U keras-tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import time
LOG_DIR = f"{int(time.time())}"
from keras.layers import BatchNormalization
from tempfile import TemporaryFile
import tensorflow.keras.regularizers as regularizers
from PIL import Image
import matplotlib.patches as patches

from google.colab import drive
drive.mount('/content/drive/')
dataset_dir = "/content/drive/My Drive/abc"


root_path = "/content/drive/My Drive/abc/"

In [ ]:
def img_preprocess():
    # Traverse all folders in the root folder
    for folder_name in os.listdir(root_path):
        # Check if it's a folder
        if os.path.isdir(os.path.join(root_path, folder_name)):
            # Print the folder path
            plane = os.path.join(folder_name)
            plane_path = plane + ("/")
            print(plane_path)

            # Create the full path to the folder containing images
            selected_path = os.path.join(root_path, plane_path)

            # Generate augmented images for images in the current folder
            images(selected_path)

def images(selected_path):
    #ImageDataGenerator object
    gen = ImageDataGenerator(
        rotation_range=360,
        width_shift_range=0,
        height_shift_range=0,
        shear_range=0,
        zoom_range=0,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    for img_ in os.listdir(selected_path):
      img = load_img(selected_path+img_)  # this is a PIL image
      x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
      x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

      # the .flow() command below generates batches of randomly transformed images
      # and saves the results to the `preview/` directory
      i = 0
      for batch in gen.flow(x, batch_size=1,save_to_dir=selected_path, save_prefix=img_, save_format='png'):
        i += 1
        if i > 20: # the number of image generation (amendments) per selected picture
           break
img_preprocess()

In [ ]:
!touch /content/drive/MyDrive/abc/AIRMODELS.txt #file creation
import os

# Create the file if it doesn't exist
if not os.path.exists("/content/drive/MyDrive/abc/AIRMODELS.txt"):
    open("/content/drive/MyDrive/abc/AIRMODELS.txt", "w").close()

# Open the file in append mode
with open("/content/drive/MyDrive/abc/AIRMODELS.txt", "a") as f:
    # Write the values to the file
    f.write("Tejas\n")
    f.write("MiG21\n")

In [ ]:
# # the google drive files
# airmodel_path = "/content/drive/My Drive/abc/MODELS.txt"
# aircraft_model = []
# f = open(airmodel_path, "r")
# print(f)
# for models in f.read().split():
#   aircraft_model.append(models.replace('"',''))
# print(len(aircraft_model))
# f.close()
# print("Selected aircraft files:",aircraft_model)
# print("Files in dataset (Google drive):",os.listdir(root_path))
import re
airmodel_path = "/content/drive/My Drive/abc/AIRMODELS.txt"
aircraft_model = []

with open(airmodel_path, "r") as f:
    for line in f:
        for word in line.split():
            aircraft_model.append(word.replace('"', ''))

print(len(aircraft_model))
print("Selected aircraft files:", aircraft_model)
print("Files in dataset (Google drive):", os.listdir(root_path))



In [ ]:
import cv2
import random
import matplotlib.pyplot as plt


training_data = []
test_data = []
RESIZE_VALUE = 130

def create_training_data():
  for type in aircraft_model:
    selected_path = root_path + type
    class_num = aircraft_model.index(type)
    for img in os.listdir(selected_path):
      try:
        image = Image.open(os.path.join(selected_path,img))
        img_array = np.array(image)
        aircraft_img = cv2.resize(img_array, (RESIZE_VALUE, RESIZE_VALUE))
        training_data.append([aircraft_img, class_num])
      except Exception as e:
        print("error:",selected_path,image)

create_training_data()
random.shuffle(training_data)
print(len(training_data))
print(training_data[0][1])
plt.imshow(training_data[0][0])


In [ ]:
import pickle

with open(root_path + 'primary_training_dataset.pkl', 'wb') as f:
    pickle.dump(training_data, f)


In [ ]:
def load_training_data():
    # Load training data into array from file
    loaded = np.load(root_path + "primary_training_dataset.pkl", allow_pickle=True)
    training_data = loaded
    for x in range(len(training_data)):
        if (training_data[x][0].shape[2] == 4):
            training_data[x][0] = cv2.cvtColor(training_data[x][0], cv2.COLOR_BGRA2BGR)
    return training_data

training_data = load_training_data()
random.shuffle(training_data)
print(len(training_data))

In [ ]:
import matplotlib.pyplot as plt

for data, label in training_data:
    plt.imshow(data)
    plt.title(f"Label: {label}")
    plt.axis('off')
    plt.show()

In [ ]:
x_train = []
y_train = []

#x_test = []
#y_test = []

#for data, label in testing_data:
#  x_test.append(data)
#  y_test.append(label)

for data, label in training_data:
  x_train.append(data)
  y_train.append(label)

x_train = np.asarray(x_train)
x_train = np.array(x_train).reshape(-1, RESIZE_VALUE, RESIZE_VALUE, 3)
y_train = np.asarray(y_train)

#x_test = np.asarray(x_test)
#x_test = np.array(x_test).reshape(-1, RESIZE_VALUE, RESIZE_VALUE, 3)
#y_test = np.asarray(y_test)

x_train = x_train / 255
#x_test = x_test / 255
print(len(x_train))
#print(len(x_test))

In [ ]:
def build_model_MAC():
    # main model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape = (RESIZE_VALUE,RESIZE_VALUE,3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))



    model.add(Flatten())

    model.add(Dense(450))
    model.add(Activation('relu'))
    model.add(Dropout(.65))

    model.add(Dense(len(aircraft_model)))
    model.add(Activation("softmax"))

    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='sparse_categorical_crossentropy',
                    optimizer=opt,
                    metrics=['acc'])

    return model

model = build_model_MAC()
epochs = 30
x = np.arange(epochs) + 1
history_ = model.fit(x_train, y_train, epochs=epochs, validation_split=.30, batch_size=64)
plt.plot(x,history_.history['acc'])
plt.plot(x,history_.history['val_acc'])
plt.xlabel("Epochs");
plt.ylabel("Accuracy")
plt.legend(['Training Accuacy', 'Cross Validation Accuracy']);

fig, ax = plt.subplots()
ax.plot(x,history_.history['loss'])
ax.plot(x,history_.history['val_loss'])
plt.xlabel("Epochs");
plt.ylabel("Loss")
plt.legend(['Training Loss', 'Cross Validation Loss']);

In [ ]:
def testSingleAircraft():
    # function takes a single image of an aircraft and classifies the aircraft category
    img = "MiG-21.11.png"
    image = Image.open(os.path.join(root_path+"/Testing_images/",img))
    img_array = np.array(image)
    aircraft_img = cv2.resize(img_array, (RESIZE_VALUE, RESIZE_VALUE))
    patch = np.expand_dims(aircraft_img, axis=0)
    plt.imshow(aircraft_img)
    # x = model.predict_classes(patch)
    x = np.argmax(model.predict(patch))
    print("True value:", img)
    print("Identified as:",aircraft_model[x[0]],"\n")
testSingleAircraft()

In [ ]:
def testSingleAircraft():
    folder_path = '/content/drive/MyDrive/abc/Testing_images'

    file_list = os.listdir(folder_path)

    for file_name in file_list:
      if file_name.endswith('.png') or file_name.endswith('.jpg'):
        image_path = os.path.join(folder_path,file_name)
        img = Image.open(image_path)
        # plt.imshow(img)
        # plt.title(file_name)
    img_array = np.array(img)
    if img_array.shape[-1] == 4:
        img_array = cv2.cvtColor(img_array, cv2.COLOR_RGBA2RGB)

    aircraft_img = cv2.resize(img_array, (RESIZE_VALUE, RESIZE_VALUE))

    patch = np.expand_dims(aircraft_img, axis=0)

    plt.imshow(aircraft_img)

    x = np.argmax(model.predict(patch))

    plt.imshow(img)
    plt.title(file_name)
    print("True Value:",file_name)
    print("Identified as:", aircraft_model[x], "\n")


testSingleAircraft()


In [ ]:
def testSingleAircraft():
    # function takes a single image of an aircraft and classifies the aircraft category
    img = "Tejas.4.png"
    image = Image.open(os.path.join(root_path+"/Testing_images/",img))
    img_array = np.array(image)
    aircraft_img = cv2.resize(img_array, (RESIZE_VALUE, RESIZE_VALUE))
    patch = np.expand_dims(aircraft_img, axis=0)
    plt.imshow(aircraft_img)
    x = np.argmax(model.predict(patch))
    print ("True value:", img)
    print("Identified as:",aircraft_model[x[0]],"\n")
testSingleAircraft()